In [1]:
import numpy.random as ra
import numpy as np
import pandas as pd

# Modelo con reversión

Vamos a simular la evolución de un tipo lineal de interés $r(t)$ a corto plazo cuya dinámica se rige por la ecuación
$$dr_t = \alpha (\mu_t - r_t) dt + \sigma dW_t,$$
donde:
* $\mu_t$ es el nivel de reversión, dependiente del tiempo
* $\alpha$ es la fuerza de la reversión
* $\sigma$ es la volatilidad del subyacente
* $W_t$ es un browniano estándar.

## Simulación para tiempos discretos
En la práctica, para simular el subyacente en tiempos discretos, para saltar desde $T=t$ a $T=t+\Delta t$ se aplica
$$r_{t + \Delta t} = r_{t} + \alpha \Delta t (\mu_t - r_t) + \sigma \sqrt{\Delta t} Z,$$
donde $Z\sim N(0, 1)$ es una normal estándar.

## Curva DF derivada

#### Realizamos una simulación para unos datos concretos:

Consideremos el tipo simple a un mes inicial $r_0=4.235\%$. Queremos simularlo mes a mes para un periodo de 10 años, sabiendo que sigue una dinámica con:
* reversión $\mu_t=r_0$ constante
* $\alpha = 0.3$
* $\sigma=0.015$

Realizamos $10^6$ simulaciones, y las empleamos para estimar la curva de factores de descuento que se deriva de este modelo. Para ello se calcula mes a mes la variable $C_t$ que representa la *cuenta corriente* resultante de invertir en $T=0$ una unidad monetaria y capitalizarla mes a mes al tipo variable $r_t$. Por ejemplo, la cuenta corriente a los 3 meses es
$$C_{3\Delta t} = 1 \cdot (1 + r_{0} \Delta t) \cdot (1 + r_{1\Delta t} \Delta t) \cdot (1 + r_{2 \Delta t} \Delta t)$$
donde $\Delta t = \frac{1}{12}$.




Entonces, el factor de descuento a un tiempo $t$ es $DF(t) = \mathbb{E}[\frac{1}{C_t}]$. Calculamos el DF para todos los tiempos $T=0, \Delta t, 2\Delta t, \dots, 10$.

In [7]:
# FUNCIONES

def simulacion_rts(r0, alpha, sigma, rng, n_meses=120, dif_t=1/12):
    """
    Objetivo: Simular una serie temporal de tipos de interés siguiendo el modelo de reversión a la media.

    Inputs:
        - r0: Tipo de interés inicial.
        - alpha: Tasa de reversión a la media (mayor alpha implica convergencia más rápida a la media).
        - sigma: Volatilidad del modelo, define la magnitud de las fluctuaciones aleatorias.
        - rng: Objeto RandomState que acutúa como semilla de las simulaciones.
        - n_meses: Número de meses a simular (por defecto, 10 años * 12 meses = 120 meses).
        - dif_t: Paso temporal (por defecto, 1/12 para meses).

    Outputs:
        - Un numpy.ndarray de valores simulados de los tipos de interés para cada mes.
    """

    # Simulamos 120 (n_meses) valores normales escalados (media=0, std=sigma*sqrt(dif_t)).
    normales = rng.normal(size=n_meses) * sigma * np.sqrt(dif_t)

    # Inicializamos la serie de tipos de interés con el valor inicial r0.
    rts = [r0]

    # Calculamos los siguientes tipos de interés usando la fórmula del modelo de reversión a la media.
    # Cada uno depende del anterior, los parametros r0, alpha y sigma y el valor aleatorio normal simulado.
    for t in range(1, n_meses + 1):
        nuevo_rt = rts[-1] + alpha * dif_t * (mu - rts[-1]) + normales[t - 1]
        rts.append(nuevo_rt)

    return np.array(rts)

def calculo_cts(array_rts, n_meses=120, dif_t=1/12):
    """
    Objetivo: Calcular las cuentas corrientes para cada mes a partir de la serie de tipos de interés.

    Inputs:
        - array_rts: Un array con los tipos de interés mensuales simulados.
        - n_meses: Número de meses considerados (por defecto, 120 meses).
        - dif_t: Paso temporal (por defecto, 1/12 para meses).

    Outputs:
        - Un numpy.ndarray de "cuentas corrientes" por mes.
    """

    # Calculamos las tasas de capitalización correspondientes a cada mes.
    tasas_cap = 1 + array_rts * dif_t 

    # Inicializamos la lista de cuentas corrientes. En el primer mes se invierte una ud monetaria.
    lista_cts = [1]

    # La cuenta corriente del mes t corresponde el producto de las tasas de capitalización hasta el mes t.
    for t in range(1, n_meses + 1):
        ct = np.prod(tasas_cap[:t])
        lista_cts.append(ct)

    return np.array(lista_cts)

def simulacion_DFs (nSims, r0, alpha, sigma, rng):
    """
    Calcula el valor esperado de los factores de descuento. Para ello simula nSims series de tipos.
    Output:
        - Un DataFrame que contiene los factores de descuento para cada mes.
    """

    # Vamos almacenando los resultados de 1/cuenta_corriente_t de cada simulacion.
    resultados_simulaciones = []
    
    for _ in range(nSims):   
        rts = simulacion_rts(r0, alpha, sigma, rng)
        cts = calculo_cts(rts)
        resultados_simulaciones.append(1/cts)

    # Convertimos la lista con todos los resultados en un DataFrame para calcular fácilmente el promedio por columna. 
    # Cada fila representa una simulación y cada columna representa un mes.
    resultados_sim = pd.DataFrame(resultados_simulaciones)
    
    # Cada columna j tiene los valores 1/c_j resultantes de todas las simulaciones. Obtenemos DF(t=j) = E[1/c_j] al hacer el promedio de la columna j.
    dfs = resultados_sim.mean(axis=0)

    return dfs

In [9]:
# Fijamos una semilla para que los resultados sean reproducibles.
rng = ra.RandomState(3452)

# Un millón de simulaciones
nSims = 10 ** 6

# Cada simulacion abarca un periodo de 10 años
n_meses = 10*12
dif_t = 1/12

# Inputs del modelo 

r0 = 0.04235
mu = r0            # En este primer caso el nivel de reversion es constante
alpha = 0.3
sigma = 0.015

dfs = simulacion_DFs (nSims, r0, alpha, sigma, rng)

# Modificamos los índices de la serie de Pandas para ver claramente cada valor a que factor de descuento corresponde.
dfs.index = ["DF(t=" + str(i) + "/" + "12)" for i in range(n_meses+1)]

dfs

DF(t=0/12)      1.000000
DF(t=1/12)      0.996483
DF(t=2/12)      0.992979
DF(t=3/12)      0.989488
DF(t=4/12)      0.986008
                  ...   
DF(t=116/12)    0.668754
DF(t=117/12)    0.666464
DF(t=118/12)    0.664181
DF(t=119/12)    0.661907
DF(t=120/12)    0.659641
Length: 121, dtype: float64

## Niveles de reversión no constantes.

Consideramos ahora medias no constantes. Renombremos como $r^1(t)$ al proceso estocástico correspondiente al tipo de interés ya descrito, que se obtiene con reversión a una media constante $\mu_t=r_0$, valor inicial $r^1(0)=r_0$ y parámetros $\alpha$ y $\sigma$.

Ahora, sea $r^2(t)$ otro proceso estocástico con la misma $\sigma$, $\alpha$ y valor inicial $r^2(0) = r_0$, pero cuyo nivel de reversión es $r_1(t)$. Y, en general, definimos el proceso estocástico $r^{k+1}$ como aquel que resulta de usar $r^{k}$ como nivel de reversión.

Vamos a simular mensualmente la evolución de los primeros 4 años. Para $k=1,\dots,20$ queremos estimar un par de parámetros de la distribución de $r^k(4)$ (es tipo a un mes tras 4 años):
* La probabilidad de que el tipo se haya vuelto negativo.
* La desviación típica muestral.

Estos estadísticos, ¿convergen cuando $k\to\infty$, o divergen?

In [21]:
# FUNCIONES

def simulacion_rts_mu_no_cte (r0, alpha, sigma, mus, rng, n_meses=48, dif_t=1/12):
    """
    Objetivo: Simular una serie temporal de tipos de interés (r_k+1) siguiendo el modelo de reversión a la media.
              En este caso la media no es constante, se emplea r_k como nivel de reversión.

    Inputs:
        - r0: Tipo de interés inicial.
        - alpha: Tasa de reversión a la media (mayor alpha implica convergencia más rápida a la media).
        - sigma: Volatilidad del modelo, define la magnitud de las fluctuaciones aleatorias.
        - mus: Niveles de reversión. Lista con el proceso estocástico r_k.
        - rng: Objeto RandomState que acutúa como semilla de las simulaciones.
        - n_meses: Número de meses a simular (por defecto, 4 años * 12 meses = 48 meses).
        - dif_t: Paso temporal (por defecto, 1/12 para meses).

    Outputs:
        - Un numpy.ndarray de valores simulados de los tipos de interés para cada mes.
    """

    # Simulamos 48 valores normales escalados (media=0, std=sigma*sqrt(dif_t))
    normales = rng.normal(size=n_meses) * sigma * np.sqrt(dif_t)
    
    # Inicializamos la lista de tipos de interés a un mes con el tipo inicial. 
    rts = [r0]

    # Calculamos los siguientes tipos de interés usando la fórmula del modelo de reversión a la media.
    # Cada uno depende del anterior, los parametros r0, alpha y sigma, r_k(t) y el valor aleatorio normal simulado.
    
    for t in range(1, n_meses+1):
        nuevo_rt = rts[-1] + alpha * dif_t * (mus[t-1] - rts[-1]) + normales[t - 1]
        rts.append(nuevo_rt)

    return np.array(rts)

def tipo_final_por_k (r0, alpha, sigma, lim_k, rng, n_meses, dif_t):
    """
    Objetivo: Simula una vez todos los procesos estocástico k's, obteniendo el tipo a un mes a los n_meses.
    Output: Array con el tipo a un mes tras n_meses (cuatro años) para cada proceso estocástico k.
    """
    
    # k = 1,..,20
    grados_reversion = range(2, lim_k+1)
    
    # Vamos guardando cada lista de niveles de reversion r^{k}, que se obtiene a partir de r^{k-1}, en una lista. 
    # Niveles_reversion es una lista de listas. [r^1(t), r^2(t), ..., r^20(t)]
    # De esta manera, el ultimo elemento guardado en esta lista, nos ayuda a calcular el siguiente, se emplea como input.
    
    rts = simulacion_rts (r0, alpha, sigma, rng, n_meses, dif_t)
    niveles_reversion = [rts]
    
    for k in grados_reversion:
        rts_k = simulacion_rts_mu_no_cte (r0, alpha, sigma, niveles_reversion[-1], rng, n_meses, dif_t)
        niveles_reversion.append(rts_k)

    # r^k (t=4) corresponde al último elemento de cada elemento de "niveles_reversion"
    # En r_ks_t4 recogemos los resultados de tipo a un mes tras cuatro años para cada grado de reversión k.

    r_ks_t4 = np.array(niveles_reversion)[:,-1]

    return r_ks_t4

In [23]:
# Fijamos una semilla para que los resultados sean reproducibles.
rng = ra.RandomState(3452)

# Simulamos 10^6 veces y creamos un dataframe con los resultados.

nSims = 10**6
lim_k = 20
n_meses=48
dif_t=1/12

resultados_simulaciones = []

for _ in range(nSims):

    r_ks_t4_sim = tipo_final_por_k (r0, alpha, sigma, lim_k, rng, n_meses, dif_t)
    resultados_simulaciones.append(r_ks_t4_sim)

# Convertimos la lista de resultados en un DataFrame para realizar calculos fácilmente por columna.
# Cada fila correspone a una simulacion, cada columna a una k. 
resultados_sim = pd.DataFrame(resultados_simulaciones)

# Probabilidad de que el tipo se haya vuelto negativo -> Frecuencia relativa de negativos por columna
frec_tipos_neg_por_k = (resultados_sim < 0).sum(axis=0) / nSims

# Desviación típica muestral -> Desviación típica por columna
desv_muestral_por_k = resultados_sim.std(axis=0)

In [25]:
frec_tipos_neg_por_k.index = frec_tipos_neg_por_k.index + 1
frec_tipos_neg_por_k.index.name = "k"

print("Probabilidad de que el tipo se haya vuelto negativo tras 4 años para cada grado de reversión k:")
print(frec_tipos_neg_por_k)

Probabilidad de que el tipo se haya vuelto negativo tras 4 años para cada grado de reversión k:
k
1     0.011215
2     0.020393
3     0.021944
4     0.022247
5     0.022092
6     0.022133
7     0.021945
8     0.022001
9     0.021759
10    0.022205
11    0.021760
12    0.021966
13    0.021693
14    0.022001
15    0.022237
16    0.021924
17    0.021875
18    0.022204
19    0.022194
20    0.022200
dtype: float64


In [27]:
desv_muestral_por_k.index = desv_muestral_por_k.index + 1
desv_muestral_por_k.index.name = "k"

print("Desviación típica muestral para cada grado de reversión k:")
print(desv_muestral_por_k)

Desviación típica muestral para cada grado de reversión k:
k
1     0.018611
2     0.020688
3     0.020974
4     0.021052
5     0.021043
6     0.021049
7     0.021047
8     0.021032
9     0.021026
10    0.021044
11    0.021004
12    0.021022
13    0.021006
14    0.021025
15    0.021069
16    0.021044
17    0.021039
18    0.021087
19    0.021062
20    0.021066
dtype: float64
